In [ ]:
import numpy as np
from matplotlib import pyplot
from IPython import display
import math
import time
import textwrap

In [ ]:
from scipy.signal import gaussian

In [ ]:
import cupy as cp

In [ ]:
def plotimg(ax, pattern):
    img = np.zeros((S,S,3), dtype=np.float32)
    img[:,:,:D] = pattern[B:-B,B:-B,...]
    ax.imshow(img)

In [ ]:
S = 128
D = 2
B = 8
BS = S + 2*B
#pattern = np.random.uniform(size=(S+2*B,S+2*B,D)).astype(np.float32)
xs, ys = np.meshgrid(*(np.arange(BS).astype(np.float32),)*2)

In [ ]:
def generate_periodic_ghost_boundaries(p,offset,size):
    #   d e | a b c d e | a b
    p[:offset,:,:] = p[size-1:offset+size-1,:,:]
    p[offset+size:,:] = p[offset:offset+offset,:]
    p[:,:offset,:] = p[:,size-1:offset+size-1,:]
    p[:,offset+size:,:] = p[:,offset:offset+offset,:]
    
def value_diff(q, p):
    delta = np.sum(np.square(q-p),axis=-1)
    # exponent is 1/2 * D / 2
    return np.power(delta, D*0.25)

def local_energy_terms(p,i,j):
    # E = sum_pairs_no_involving_(x',y') f(x,y) + sum_over_pairs_involving_x'or_y' f(x,y)
    #   =         ...    sum_x f(x,y') + sum_y (x',y)
    sigma_i = 2.1
    sigma_s = 1.
    one_over_sigma_i_sqr = 1./sigma_i**2
    one_over_sigma_s_sqr = 1./sigma_s**2
    pos_terms = -(np.square(ys-ys[i,j]) + np.square(xs-xs[i,j]))*one_over_sigma_i_sqr 
    val_terms = -value_diff(p,p[i,j][np.newaxis,np.newaxis,...])*one_over_sigma_s_sqr
    return pos_terms, val_terms

def local_energy(p, i,j):
    p, v = local_energy_terms(p,i,j)
    return np.sum(np.exp(p + v))

def energy_change_term(p, ij1, ij2):
    e1 = local_energy(p, *ij1)
    e2 = local_energy(p, *ij2)
    return e1+e2

def swap(p, ij1, ij2):
    i,j = ij1
    s,t = ij2
    tmp = p[i,j].copy()
    p[i,j] = p[s,t]
    p[s,t] = tmp

# def energy_change_of_swap(p, ij1, ij2):
#     before1 = local_energy(p, *ij1)
#     before2 = local_energy(p, *ij2)
#     swap(p, ij1, ij2)
#     # FIXME: regenerate boundary??!
#     after1 = local_energy(p, *ij1)
#     after2 = local_energy(p, *ij2)
#     swap(p, ij1, ij2) # swap back
#     return after1+after2-before1-before2

In [ ]:
local_energy_kernel = cp.RawKernel(textwrap.dedent(r"""
extern "C" __device__
float sqr(float x) { return x*x; }

extern "C" __global__
void local_energy(const float* pattern, int w, int h, int dims, int yref, int xref, float* terms)
{
    const float one_over_sigma_i_sqr = 1.f/(2.1f*2.1f);
    const float one_over_sigma_s_sqr = 1.f;

    const int x = blockDim.x * blockIdx.x + threadIdx.x;
    const int y = blockDim.y * blockIdx.y + threadIdx.y;

    if (x < w && y < h)
    {
        const float dxy = (sqr(x-xref) + sqr(y-yref))*one_over_sigma_i_sqr;
        float val = 0.f;
        for (int d=0; d<dims; ++d)
        {
            val += sqr(pattern[(y*w+x)*dims+d] - pattern[(yref*w+xref)*dims+d]);
        }
        val = powf(val, dims*0.25f)*one_over_sigma_s_sqr;
        terms[y*w+x] = expf(-val - dxy);
    }
}
"""),'local_energy')


boundary_kernel1 = cp.RawKernel(r"""
#define OFFSET(i,j) \
  (((j)*w+(i))*dims)
#define CPY(dst, src) \
    do { for (int d=0; d<dims; ++d) pattern[dst+d] = pattern[src+d]; } while(false)

extern "C" __global__
void boundary_kernel1(float* pattern, int w, int dims, int b, int s)
{
    int i = threadIdx.x;
    int j = blockDim.y * blockIdx.y + threadIdx.y;
    if (j < s && i < b)
    {
        {
            const int src = OFFSET(s+i, j+b);
            const int dst = OFFSET(i, j+b);
            CPY(dst, src);
        }
        {
            const int src = OFFSET(b+i, j+b);
            const int dst = OFFSET(s+b+i, j+b);
            CPY(dst, src);
        }
    }
}
""", "boundary_kernel1")


boundary_kernel2 = cp.RawKernel(r"""
#define OFFSET(i,j) \
  (((j)*w+(i))*dims)
#define CPY(dst, src) \
    do { for (int d=0; d<dims; ++d) pattern[dst+d] = pattern[src+d]; } while(false)

extern "C" __global__
void boundary_kernel2(float* pattern, int w, int dims, int b, int s)
{
    int j = threadIdx.x;
    int i = blockDim.y * blockIdx.y + threadIdx.y;
    if (j < s && i < w)
    {
        {
            const int src = OFFSET(i, j+s);
            const int dst = OFFSET(i, j);
            CPY(dst, src);
        }
        {
            const int src = OFFSET(i, j+b);
            const int dst = OFFSET(i, j+s+b);
            CPY(dst, src);
        }
    }
}
""", "boundary_kernel2")


def cuda_energy_change_term(d_pattern, d_tmp, ij1, ij2):
    blocks = 8
    assert ((BS%blocks)==0)
    local_energy_kernel((blocks,blocks),(BS//blocks,BS//blocks), (d_pattern, BS, BS, D, ij1[0], ij1[1], d_tmp))
    e1 = cp.sum(d_tmp)
    local_energy_kernel((blocks,blocks),(BS//blocks,BS//blocks), (d_pattern, BS, BS, D, ij2[0], ij2[1], d_tmp))
    e2 = cp.sum(d_tmp)
    return e1+e2

def cuda_generate_periodic_ghost_boundaries(p):
    boundary_kernel1((1,S),(B,1),(p, BS, D, B, S))
    boundary_kernel2((1,BS),(B,1),(p, BS, D, B, S))

# @cuda.jit
# def swap_kernel(pattern, y0, x0, y1, x1):
#     d = cuda.threadIdx.x
#     tmp = pattern[y0,x0,d]
#     pattern[y0,x0,d] = pattern[y1,x1,d]
#     pattern[y1,x1,d] = tmp

# def cuda_swap(d_pattern, ij1, ij2):
#     swap_kernel[1, D](d_pattern, *ij1, *ij2)

Code Generation
------------------------

In [ ]:
data = np.load('/mnt/scratch/best.npy')
fig, ax = pyplot.subplots(1,1, figsize=(10,10))
plotimg(ax,data)
data = data[B:-B,B:-B,...]
pyplot.show()

In [ ]:
print(f"// shape = {data.shape}")
print("{")
cpprows = []
for row in range(data.shape[0]):
    cpprow = []
    for col in range(data.shape[1]):
        if data.shape[2] == 1:
            el = str(data[row,col])
        else:
            el = '{'+','.join(str(d) for d in data[row,col])+'}'
        cpprow.append(el)
    cpprows.append(','.join(cpprow))
elems = map(lambda s: '  '+s, cpprows)
print(',\n'.join(elems))
print("}")

In [ ]:
img = np.tile(data, [3,3,1])
fig, ax = pyplot.subplots(1,1,figsize=(25,25))
ax.imshow(img[...,0])
pyplot.show()